In [1]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

### 1. Formulate the statement of the interpolation problem with Cubic Spline [mathematical formula]

Given a set of $n + 1$ data points  $(x_{i},y_{i})$ where $x_{i} \ne x_{j}$ for all $i,j$ : $i\ne j$.  
$a=x_{0}<x_{1}<...x_{n-1}<x_{n}=b$  
$f(x_{i}) = y_{i}$  
The cubic spline function $S(x)$ have to satisfy following properties:  
1) $S(x) \in C^2[a;b]$  
2) on each subinterval $[x_{i-1};x_{i}]$ $S_{i}(x) = ax^3+bx^2+cx+d$ where $i = 1..n$. $S(x) = \bigcup_{i=0}^{n} S_{i}(x)$  
3) $S(x_{i}) = y_{i} $





### 2. Formulate the functional and differential compatibility conditions [mathematical formula]

1) $S_{i}(x_{i})=f(x_{i})$  
2) $S_{i}(x_{i+1})=f(x_{i+1})$  
3) $S''_{i}(x_{i})=f''(x_{i})$  
4) $S''_{i+1}(x_{i+1})=f''(x_{i+1})$

### 3. Formulate stitching conditions [mathematical formula]



$ S'_{i-1}(x)|_{x=x_{i}}=S'_{i}(x)|_{x=x_{i}} $










### 4. Justify why these conditions provide you with the required smoothness [thesis text, no more than 500 characters]

$S_{i}(x_{i})=f(x_{i})$  and  $S_{i}(x_{i+1})=f(x_{i+1}) \implies$ $S_{i-1}(x_{i})=f(x_{i})=S_{i}(x_{i})$ -guarantee continuity of function. Since values of functions are the same in grid's nodes and functions are continuies in interval $[x_{i-1};x_{i}]$ because the are cubic functions. 

$S''_{i}(x_{i})=f''(x_{i})$  and $S''_{i+1}(x_{i+1})=f''(x_{i+1})$ - guarantee continuty of second derivative of function. 

$ S'_{i-1}(x)|_{x=x_{i}}=S'_{i}(x)|_{x=x_{i}} $ - guarantee continuty of first derivative of function. 
















### 5. Derive dependency formula: the dependence of the second derivatives at the grid nodes on the increment of the function (the function values difference on the grid nodes). [Mathematical formulas derivation. Detailed, with clear transitions]

### YOUR TEXT/CODE HERE
<img src ="1.jpg"  />
<img src ="2.jpg" />










### 6. Create a system of equations using this formula [Matrix representation. Mathematical formulas]


<img src ="3.jpg" />







### 7. Explain what is an unknown variable in this system. whether the system is closed with respect to an unknown variable. What is missing for closure. [Text, no more than 200 characters]

$m_{1}$ and $m_{n}$ had to be unknown, but in task 6 I made them zeroes 

### 8. Bring this matrix to the appropriate form to use the Tridiagonal matrix algorithm [Mathematical derivation. Use Gauss Elimination]

matrix in task 6 is already triagonal matrix.



### 9. Derive formulas of direct pass and reverse pass of Tridiagonal matrix algorithm [Mathematical formals]

Assume we have system of equations of following type:
$$\begin{bmatrix}  
b_1 & c_1 & 0 & 0 & ... & 0 \\ 
a_2 & b_2 & c_2 & 0 & ... & 0 \\ 
0 & a_3 & b_3 & c_3 & 0 & 0 \\ 
. & . &  &  &  & . \\ 
. & . &  &  &  & . \\ 
. & . &  &  &  & c_{k-1} \\ 
0 & 0 & 0 & 0 & a_k & b_k \\ 
\end{bmatrix} \begin{bmatrix}  
f_1 \\ 
f_2 \\ 
f_3 \\ 
.\\ 
.\\ 
.\\ 
f_k \\ 
\end{bmatrix} = \begin{bmatrix} 
d_1 \\ 
d_2 \\ 
d_3 \\ 
.\\ 
.\\ 
.\\ 
d_k \\ 
\end{bmatrix}$$  
Using Gauss Elimantion we want to have matrix of follwong type
$$
\begin{bmatrix}  
1 & c^{*}_1 & 0 & 0 & ... & 0 \\ 
0 & 1 & c^{*}_2 & 0 & ... & 0 \\ 
0 & 0 & 1 & c^{*}_3 & 0 & 0 \\ 
. & . &  &  &  & . \\ 
. & . &  &  &  & . \\ 
. & . &  &  &  & c^{*}_{k-1} \\ 
0 & 0 & 0 & 0 & 0 & 1 \\ 
\end{bmatrix} \begin{bmatrix}  
f_1 \\ 
f_2 \\ 
f_3 \\ 
.\\ 
.\\ 
.\\ 
f_k \\ 
\end{bmatrix} = \begin{bmatrix} 
d^{*}_1 \\ 
d^{*}_2 \\ 
d^{*}_3 \\ 
.\\ 
.\\ 
.\\ 
d^{*}_k \\ 
\end{bmatrix}
$$ 
Coefficents  $c^{*}_{i}$ and  $d^{*}_{i}$ can be computer following:

$$
c^{*}_i = \left\{
     \begin{array}{lr}
       \frac{c_1}{b_1} & ; i = 1\\
       \frac{c_i}{b_i - c^{*}_{i-1} a_i} & ; i = 2,3,...,k-1
     \end{array}
   \right.
$$

$$
d^{*}_i = \left\{
     \begin{array}{lr}
       \frac{d_1}{b_1} & ; i = 1\\
       \frac{d_i-d^{*}_{i-1} a_i}{b_i - c^{*}_{i-1} a_i} & ; i = 2,3,...,k-1
     \end{array}
   \right.
$$
So solution is:
$f_k = d^{*}_k, \qquad f_i = d^{*}_i - c^{*}_i*f_{i+1} \qquad i = k-1, k-2, ... ,2,1$

### 10. Implement code prototype of the future algorithm implementation. Classes/methods (if you use OOP), functions. The final implementation (on language chosen by you) should not differ from the functions declared in the prototype. [Python code]

In [1]:
### YOUR CODE HERE

def build_matrix(data_points):
    pass

def solve_triadiagonal_matrix(matrix):
    pass
class polynom:
    def __init__(self,m_s,points):
        self.ms = m_s
        self.points = points
        pass
    def build():
        pass
    def compute(points):
        pass

    
def main():
    pass
    








### 11. Derive formula of Cubic Spline method error [Mathematical formulas]


<img src ="4.jpg" />













### 12. Rate the complexity of the algorithm [Text, and rate in terms of big O, no more than 100 characters]

### YOUR TEXT/CODE HERE
1) The most complicated operation is to solve matrix. Since it is tridiagonal solving can be done in $O(n)$







### Congrats!